# 0.0 - Random Forest

Os parâmetros utilizados para treinar o algoritmo **Random Forest** serão a profundidade máxima da árvore (`max_depth`) e o número máximo de estimadores (árvores na florestas), o `n_estimators`.</br>Os valores padrão destes parâmetros são `None` e `100`, respectivamente. Sobre o `max_depth`, a documentação oficial diz o seguinte:</br></br>https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html</br></br>*"The `maximum depth` of the tree. If `None`, then nodes are expanded until all leaves are pure or until all leaves contain less than `min_samples_split` samples"*.</br></br>*Por padrão, a árvore se expande até que todas as folhas sejam puras ou até que contenham um número de amostras menor que o valor default do parâmetro `min_samples_split`, 2 amostras*.</br></br>
Para os cálculos das métricas de performance, considerei a classe positiva como sendo a **0**.</br>Por padrão, o *sklearn* toma como sendo a classe **1**.

# 1.0 - Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics as mt
from itertools import product

from matplotlib import pyplot as plt

# 2.0 - Carregando os dados

In [2]:
# Treino
X_train = pd.read_csv('train/X_training.csv')
y_train = pd.read_csv('train/y_training.csv')

# Validação
X_val = pd.read_csv('validation/X_validation.csv')
y_val = pd.read_csv('validation/y_validation.csv')

# Teste
X_test = pd.read_csv('test/X_test.csv')
y_test = pd.read_csv('test/y_test.csv')

# 3.0 - Funções

In [3]:
def metrics(y, y_pred, step, algorithm):
    """
    Calcula várias métricas de avaliação de um modelo de classificação e as retorna em um DataFrame.

    Parâmetros:
    -----------
    y : array-like
        Valores da variável resposta.
    
    y_pred : array-like
        Valores preditos pelo modelo.
    
    step : str
        Etapa do processo de modelagem (ex: 'treino', 'validação', 'teste').
    
    algorithm : str
        Nome do algoritmo usado.

    Retorno:
    --------
    df_metrics : pandas.DataFrame
        DataFrame contendo as métricas calculadas:
        - 'Algorithm': Nome do algoritmo.
        - 'Step': Etapa do processo (treino, validação ou teste).
        - 'Accuracy': Valor da Acurácia.
        - 'Precision': Valor da Precision.
        - 'Recall': Valor da Recall.
        - 'F1-Score': Valor de F1-Score.
    """
    # Calcula as métricas
    acuracia  = mt.accuracy_score( y, y_pred )
    precision = mt.precision_score( y, y_pred, pos_label = 0 )
    recall    = mt.recall_score( y, y_pred, pos_label = 0 )
    f1        = mt.f1_score( y, y_pred, pos_label = 0 )

    # Cria o DataFrame com as métricas
    df_metrics = pd.DataFrame({   
        'Algorithm': [algorithm],
        'Step': [step],
        'Accuracy': [np.round(acuracia, 4)],
        'Precision': [np.round(precision, 4)],
        'Recall': [np.round(recall, 4)],
        'F1-Score': [np.round(f1, 4)]
    })
    
    return df_metrics

def exporta_excel(nome_algoritmo):
    """
    Exporta os DataFrames de métricas para arquivos Excel em diferentes pastas.

    Args:
        nome_algoritmo (str): Nome do algoritmo utilizado.

    Returns:
        None
    """
    etapas = ['train', 'validation', 'test']
    df_list = [df_metrics_train, df_metrics_val, df_metrics_test]

    for etapa, df in zip(etapas, df_list):
        df.to_excel(f'{etapa}/metrics/metrics_{etapa}_{nome_algoritmo}.xlsx', index=False)

# 4.0 - Ajustando os dados

In [4]:
# Como o atributo 'id' não contribui para o aprendizado do modelo, iremos retirá-lo.
X_train, X_val, X_test = [ df.drop(columns = ['id']) for df in [X_train, X_val, X_test] ]

# renomeia a coluna '0' da variável resposta com o nome original 'satisfaction'
y_train, y_val, y_test = [ df.rename(columns = {'0': 'satisfaction'}) for df in [y_train, y_val, y_test] ]

# transformação do formato DataFrame para Series, essencialmente um array unidimensional
y_train, y_val, y_test = [ df.loc[:, 'satisfaction'] for df in [y_train, y_val, y_test] ]

# 5.0 - Desempenho do modelo

## 5.1 - Dados de treino

In [8]:
# Definição do modelo
random_forest_train = RandomForestClassifier( random_state = 0 )

# treinamento do algoritmo. 
random_forest_train.fit(X_train, y_train)

# Predição sobre os dados de treino
y_pred_train = random_forest_train.predict(X_train)

# Métricas
df_metrics_train = metrics(y_train, y_pred_train, 'Train', 'Random Forest')
df_metrics_train

,Algorithm,Step,Accuracy,Precision,Recall,F1-Score
0,Random Forest,Train,1.0,1.0,1.0,1.0


## 5.2 - Dados de validação

In [9]:
# definindo o modelo
random_forest_val = RandomForestClassifier( random_state = 0 )

# treinamento do algoritmo. 
random_forest_val.fit(X_train, y_train)

# Predição sobre os dados de validação
y_pred_val = random_forest_val.predict(X_val)

# Métricas
df_metrics_val = metrics(y_val, y_pred_val, 'Validation', 'Random Forest')
df_metrics_val

,Algorithm,Step,Accuracy,Precision,Recall,F1-Score
0,Random Forest,Validation,0.9622,0.9561,0.9783,0.967


## 5.3 - Ajuste fino dos hiperparâmetros (Etapa de *Fine Tuning*)

In [14]:
# array para os valores testados de 'max_depth' (máxima profundidade da arvore)
max_depth_array = np.arange(2, 31, 1)

# lista para o número de estimadores (árvores da floresta)
n_estimators_list = [100, 500, 1000]

# Criar DataFrame para armazenar resultados
metrics_df = pd.DataFrame(columns=['max_depth', 'n_estimators', 'accuracy', 'precision', 'recall', 'f1_score'])

# Usar 'itertools.product' para combinar 'max_depth' e 'n_estimators'
for m, n in product(max_depth_array, n_estimators_list):
    # Definir e treinar o modelo RandomForest
    model = RandomForestClassifier(max_depth=m, n_estimators=n, random_state=0)
    model.fit(X_train, y_train)

    # Predição sobre os dados de validação
    y_pred = model.predict(X_val)

    # Calcular métricas de performance
    acuracia = mt.accuracy_score(y_val, y_pred)
    precision = mt.precision_score(y_val, y_pred, pos_label=0)
    recall = mt.recall_score(y_val, y_pred, pos_label=0)
    f1 = mt.f1_score(y_val, y_pred, pos_label=0)

    # Armazenar os resultados em uma nova linha como DataFrame
    new_row = pd.DataFrame({
        'max_depth': [m],
        'n_estimators': [n],
        'accuracy': [acuracia],
        'precision': [precision],
        'recall': [recall],
        'f1_score': [f1]
    })

    # Concatenar a nova linha ao DataFrame existente
    metrics_df = pd.concat([metrics_df, new_row], ignore_index=True)

In [15]:
# DataFrame com os valores das métricas de performance para os valores de 'max_depth' e 'n_estimators' testados
metrics_df

,max_depth,n_estimators,accuracy,precision,recall,f1_score
0,2,100,0.865118,0.860505,0.909370,0.884263
1,2,500,0.875833,0.867536,0.921579,0.893741
2,2,1000,0.876251,0.867943,0.921863,0.894090
3,3,100,0.899096,0.895335,0.930721,0.912685
4,3,500,0.897069,0.893206,0.929472,0.910978
...,...,...,...,...,...,...
82,29,500,0.962998,0.956765,0.978932,0.967722
83,29,1000,0.962579,0.956025,0.978989,0.967371
84,30,100,0.962418,0.956570,0.978081,0.967206
85,30,500,0.962933,0.956710,0.978876,0.967666


Como o objetivo principal da pesquisa é identificar os clientes insatisfeitos para tomar ações corretivas, o **recall** é a figura de mérito mais relevante.</br>Assim, um alto **recall** garante que a maioria dos clientes insatisfeitos seja corretamente identificada.</br>
Os melhores valores de **max_depth** e **n_estimators** são aqueles que produzem o maior valor da métrica de interesse.

In [16]:
metrics_df.sort_values(by='recall', ascending=False).head(1)

,max_depth,n_estimators,accuracy,precision,recall,f1_score
86,30,1000,0.963191,0.956628,0.979443,0.967901


In [11]:
# Melhores parâmetros: 'max_depth' = 30 e 'n_estimators' = 1000
best_max_depth = 30
best_n_estimators = 1000

## 5.4 - Desempenho do modelo para os dados de teste

In [12]:
# Definição do modelo
random_forest_test = RandomForestClassifier( 
                                            max_depth=best_max_depth, 
                                            n_estimators=best_n_estimators, 
                                            random_state=0
)

# Juntar os dados de treino e validação
random_forest_test.fit( pd.concat( [X_train, X_val] ),
                        pd.concat( [y_train, y_val] ) )

# Predições sobre os dados de teste
y_pred_test = random_forest_test.predict( X_test )

# Métricas
df_metrics_test = metrics(y_test, y_pred_test, 'Test', 'Random Forest')
df_metrics_test

,Algorithm,Step,Accuracy,Precision,Recall,F1-Score
0,Random Forest,Test,0.9631,0.9567,0.9785,0.9675


# 6.0 - Gera as planilhas com os resultados

In [13]:
exporta_excel('random_forest')